In [1]:
import folium, random

random.seed('0123456')

MUC_COORD = [48.13710, 11.57561]

tilesetstyles = ['cartodbpositron', 'stamenterrain', 'OpenStreetMap']
tileset = tilesetstyles[1]

maps = {}
for i in ["beeline", "real"]:
    maps[i] = folium.Map(location=MUC_COORD, zoom_start=11, tiles=tileset, width=640, height=480)

latitude_min = 48.2050
latitude_max = 48.0562
longitude_min = 11.4153
longitude_max = 11.7664

depot = [48.2094, 11.5782]
[folium.Marker(depot, icon=folium.Icon(color='green')).add_to(maps[m]) for m in maps.keys()]

K = 2
Q = 7
T = 100000

nCustomers = 10
customers = []
b = [0]
for i in range(nCustomers):
    customers.append([
        random.uniform(latitude_min, latitude_max), 
        random.uniform(longitude_min, longitude_max)])
    b.append(1)

locations = [depot] + customers
nLocations = len(locations)

[folium.Marker(c).add_to(maps[m]) for c in customers for m in maps.keys()]

maps["beeline"]

### Distance Calculation

#### Option 1: Euclidean distance in km based on latitude and longitude considering earth curvature (taken from https://en.kompf.de/gps/distcalc.html)

In [2]:
from math import sin, cos, sqrt, atan2, radians

distance = {}

R = 6373.0 # radius of earth
for i in range(nLocations):
    c0 = locations[i]
    lat0 = radians(c0[0])
    lon0 = radians(c0[1])
    for j in range(nLocations):
        c1 = locations[j]
        lat1 = radians(c1[0])
        lon1 = radians(c1[1])

        dlat = lat1 - lat0
        dlon = lon1 - lon0

        a = sin(dlat / 2) ** 2 + cos(lat0) * cos(lat1) * sin(dlon / 2) ** 2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        # distance in km
        distance[i, j] = R * c

for i in range(nLocations):
    for j in range(nLocations):
        print(str(round(distance[i, j], 2))+"\t", end=" ")
    print()

0.0	 9.09	 16.45	 11.15	 4.16	 18.76	 11.61	 9.8	 10.65	 18.51	 5.35	 
9.09	 0.0	 20.5	 16.15	 12.33	 12.74	 7.92	 17.58	 14.6	 22.94	 7.82	 
16.45	 20.5	 0.0	 5.33	 13.04	 19.25	 14.65	 9.97	 6.07	 2.47	 13.13	 
11.15	 16.15	 5.33	 0.0	 7.73	 17.96	 11.83	 5.83	 2.03	 7.38	 8.41	 
4.16	 12.33	 13.04	 7.73	 0.0	 19.68	 12.38	 5.66	 7.84	 14.89	 5.84	 
18.76	 12.74	 19.25	 17.96	 19.68	 0.0	 7.31	 22.62	 15.94	 21.49	 13.94	 
11.61	 7.92	 14.65	 11.83	 12.38	 7.31	 0.0	 15.72	 9.83	 17.11	 6.63	 
9.8	 17.58	 9.97	 5.83	 5.66	 22.62	 15.72	 0.0	 7.35	 11.09	 10.14	 
10.65	 14.6	 6.07	 2.03	 7.84	 15.94	 9.83	 7.35	 0.0	 8.42	 7.07	 
18.51	 22.94	 2.47	 7.38	 14.89	 21.49	 17.11	 11.09	 8.42	 0.0	 15.48	 
5.35	 7.82	 13.13	 8.41	 5.84	 13.94	 6.63	 10.14	 7.07	 15.48	 0.0	 


#### Option 2: real-world distances (car, bike, or foot)

In [6]:
import requests

def get_real_distance(origin: dict, destination: dict, mode: str = 'car'):
    lat_from = origin[0]
    long_from = origin[1]
    lat_to = destination[0]
    long_to = destination[1]

    url = 'http://router.project-osrm.org/route/v1/'+ mode +\
        '/{},{};{},{}?alternatives=false&steps=true&geometries=geojson&overview=full&annotations=true'.format(
            long_from, lat_from, long_to, lat_to)
    return requests.get(url).json().get('routes')[0].get('distance')

distance = {}
for i in range(nLocations):
    for j in range(nLocations):
        distance[i, j] = get_real_distance(locations[i], locations[j])/1000
        print(str(round(distance[i, j], 2))+"\t", end=" ")
    print()

0.0	 22.72	 34.06	 27.61	 6.11	 24.1	 15.16	 17.02	 13.58	 27.6	 7.32	 
21.44	 0.0	 48.64	 42.19	 26.63	 18.51	 10.92	 31.6	 19.84	 42.18	 10.46	 
32.17	 47.49	 0.0	 12.83	 30.93	 36.6	 27.57	 22.71	 10.55	 6.2	 18.6	 
28.35	 23.3	 20.85	 0.0	 13.95	 25.8	 16.77	 10.33	 2.79	 14.39	 13.32	 
6.91	 27.45	 31.54	 25.09	 0.0	 25.26	 16.33	 14.51	 12.26	 25.09	 6.9	 
25.51	 19.32	 37.21	 24.45	 25.56	 0.0	 10.26	 31.81	 21.51	 41.14	 18.11	 
15.85	 10.88	 27.66	 14.9	 15.9	 9.7	 0.0	 22.27	 11.96	 31.59	 8.44	 
16.11	 31.44	 22.28	 9.76	 8.53	 32.11	 23.08	 0.0	 11.77	 15.82	 19.56	 
14.55	 20.07	 10.65	 2.96	 10.95	 20.93	 11.91	 11.92	 0.0	 15.97	 10.1	 
27.13	 42.46	 6.22	 14.54	 25.9	 38.2	 29.18	 17.68	 16.55	 0.0	 24.42	 
8.01	 10.62	 30.55	 12.68	 6.9	 17.68	 8.75	 18.93	 9.87	 24.1	 0.0	 


In [3]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model()

x = {}
for i in range(nLocations):
    for j in range(nLocations):
        for k in range(K):
            x[i, j, k] = model.addVar(vtype=GRB.BINARY)
y = {}
for i in range(nLocations):
    for k in range(K):
        y[i, k] = model.addVar(vtype=GRB.BINARY)
z = {}
for i in range(nLocations):
    z[i] = model.addVar(vtype=GRB.CONTINUOUS, lb = 0)

# each location is reached
for j in range(1, nLocations):
    for k in range(K):
        model.addConstr(gp.quicksum(x[i, j, k] for i in range(nLocations)) == y[j, k])
# each location is left
for i in range(1, nLocations):
    for k in range(K):
        model.addConstr(gp.quicksum(x[i, j, k] for j in range(nLocations)) == y[i, k])
for i in range(nLocations):
    model.addConstr(gp.quicksum(x[i, i, k] for k in range(K)) == 0)
# sub-tour elimination
for i in range(1, nLocations):
    for j in range(1, nLocations):
        if not (i == j):
            model.addConstr(z[i] - z[j] + nLocations * gp.quicksum(x[i, j, k] for k in range(K)) <= nLocations - 1)
# cluster
for i in range(1, nLocations):
    model.addConstr(gp.quicksum(y[i, k] for k in range(K)) == 1)
model.addConstr(gp.quicksum(y[0, k] for k in range(K)) == K)
# capacities
for k in range(K):
    model.addConstr(gp.quicksum(b[i] * y[i, k] for i in range(1, nLocations)) <= Q)
    model.addConstr(gp.quicksum(distance[i, j] * x[i, j, k] for i in range(nLocations) for j in range(nLocations)) <= T)
            
model.setObjective(
    gp.quicksum(x[i, j, k]*distance[i, j] for i in range(nLocations) for j in range(nLocations) for k in range(K)), 
    GRB.MINIMIZE
)
            
model.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-14
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 156 rows, 275 columns and 1124 nonzeros
Model fingerprint: 0xdd2bfc1d
Variable types: 11 continuous, 264 integer (264 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Found heuristic solution: objective 179.1677400
Presolve removed 24 rows and 35 columns
Presolve time: 0.00s
Presolved: 132 rows, 240 columns, 820 nonzeros
Variable types: 10 continuous, 230 integer (230 binary)

Root relaxation: objective 5.329918e+01, 68 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 

### Visualization

#### Option 1: bee line

In [4]:
list_colors = ["#d11141", "#00b159", "#00aedb", "#f37735", "#ffc425"]

for k in range(K):
    for i in range(nLocations):
        for j in range(nLocations):
            if x[i, j, k].x == 1.0:
                a = locations[i]
                b = locations[j]
                folium.PolyLine([a, b], color=list_colors[k]).add_to(maps["beeline"])

maps["beeline"]

#### Option 2: real world routes

In [5]:
import requests
import pandas as pd

def get_geo_loc_route(origin: dict, destination: dict, mode: str = 'car'):
    lat_from = origin[0]
    long_from = origin[1]
    lat_to = destination[0]
    long_to = destination[1]

    url = 'http://router.project-osrm.org/route/v1/'+mode +\
        '/{},{};{},{}?alternatives=false&steps=true&geometries=geojson&overview=full&annotations=true'.format(
            long_from, lat_from, long_to, lat_to)
    response = requests.get(url)
    route_geo = response.json().get('routes')[0].get(
        'geometry').get('coordinates')
    route_df = pd.DataFrame(route_geo, columns=['lon', 'lat'])
    dist = response.json().get('routes')[0].get(
        'legs')[0].get('annotation').get('distance')
    dist.insert(len(dist), 0)
    route_df['dist_to_next'] = dist

    return route_df

for k in range(K):
    for i in range(nLocations):
        for j in range(nLocations):
            if x[i, j, k].x == 1.0:
                route = get_geo_loc_route(locations[i], locations[j])
                for r in range(1, len(route)):
                    lon0 = route.get("lon")[r-1]
                    lat0 = route.get("lat")[r-1]
                    lon1 = route.get("lon")[r]
                    lat1 = route.get("lat")[r]
                    folium.PolyLine([[lat0, lon0], [lat1, lon1]], color=list_colors[k]).add_to(maps["real"])

maps["real"]